# CycleGAN
CycleGAN Implementation.

#### References
* [Paper](https://arxiv.org/pdf/1611.07004.pdf)
* [Confluece page](https://machinereinforcedbook.atlassian.net/wiki/spaces/ML/pages/22052990/Pix2Pix)
* [Medium Article](https://medium.com/datadriveninvestor/style-transferring-of-image-using-cyclegan-3cc7aff4fe61)
* [Medium Article](https://blog.paperspace.com/unpaired-image-to-image-translation-with-cyclegan/)

In [21]:
import sys
sys.path.insert(0,'..')
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
from PIL import Image
import itertools
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from dataset_utils.img_folder_dataset_cycle import ImageDataset
from torch.optim.lr_scheduler import StepLR
import numpy as np
from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)
num_gpu = torch.cuda.device_count()
#num_gpu = 8
print('Number of GPUs Available:', num_gpu)
print('Pytorch version:', torch.__version__)

# Tensorboard
from torch.utils.tensorboard import SummaryWriter
!rm -rf ./runs
writer = SummaryWriter('./runs/train')

# Metaparameters
learning_rate = 0.0002
b1 = 0.5
b2 = 0.999
img_height = img_width = 256
n_residual_blocks = 9
dataset_name = 'maps'
batch_size = 5
num_epochs = 200
# Loss weight of L1 pixel-wise loss between translated image and real image
lambda_pixel = 1000

# Calculate output of image discriminator (PatchGAN)
patch = (1, img_height // 2 ** 4, img_width // 2 ** 4)
print('Patch size:', patch)

invert_targets = True

Device: cuda:0
Number of GPUs Available: 8
Pytorch version: 1.2.0
Patch size: (1, 16, 16)


#### DataLoaders and Replay Buffer

The replay Buffer will bring back previously generated samples, this is useful on the Forward/Backward cycles.

In [22]:
class ReplayBuffer:
    def __init__(self, max_size=50):
        assert max_size > 0, "Empty buffer or trying to create a black hole. Be careful."
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0, 1) > 0.5:
                    i = random.randint(0, self.max_size - 1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))
    

# Image transformations
transforms_ = [
    transforms.Resize(int(img_height * 1.12), Image.BICUBIC),
    transforms.RandomCrop((img_height, img_width)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

# Training data loader
dataloader = DataLoader(
    ImageDataset("../data/%s" % dataset_name, transforms_=transforms_, unaligned=True),
    batch_size=batch_size,
    shuffle=True,
    num_workers=10,
)
# Test data loader
val_dataloader = DataLoader(
    ImageDataset("../data/%s" % dataset_name, transforms_=transforms_, unaligned=True, mode="test"),
    batch_size=5,
    shuffle=True,
    num_workers=1,
)

ValueError: num_samples should be a positive integer value, but got num_samples=0

#### Declare Generator and Discriminator Models

In [12]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.InstanceNorm2d(in_features),
        )

    def forward(self, x):
        return x + self.block(x)


class GeneratorResNet(nn.Module):
    def __init__(self, input_shape, num_residual_blocks=9):
        super(GeneratorResNet, self).__init__()

        channels = input_shape[0]

        # Initial convolution block
        out_features = 64
        model = [
            nn.ReflectionPad2d(channels),
            nn.Conv2d(channels, out_features, 7),
            nn.InstanceNorm2d(out_features),
            nn.ReLU(inplace=True),
        ]
        in_features = out_features

        # Downsampling
        for _ in range(2):
            out_features *= 2
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

        # Residual blocks
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(out_features)]

        # Upsampling
        for _ in range(2):
            out_features //= 2
            model += [
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

        # Output layer
        model += [nn.ReflectionPad2d(channels), nn.Conv2d(out_features, channels, 7), nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


##############################
#        Discriminator
##############################


class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()

        channels, height, width = input_shape

        # Calculate output shape of image discriminator (PatchGAN)
        self.output_shape = (1, height // 2 ** 4, width // 2 ** 4)

        def discriminator_block(in_filters, out_filters, normalize=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )

    def forward(self, img):
        return self.model(img)

    
input_shape = (3, img_height, img_width)
G_AB = GeneratorResNet(input_shape, n_residual_blocks)
G_BA = GeneratorResNet(input_shape, n_residual_blocks)
D_A = Discriminator(input_shape)
D_B = Discriminator(input_shape)

#### Define Loss Functions

In [13]:
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

#### Define Optimizers

In [15]:
# Optimizers
optimizer_G = torch.optim.Adam(
    itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=learning_rate, betas=(b1, b2)
)
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=learning_rate, betas=(b1, b2))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=learning_rate, betas=(b1, b2))

#### Train loop

In [ ]:
for epoch in tqdm(range(num_epochs)):
    running_loss_G = 0.0
    running_loss_D = 0.0
    running_pixelwise_loss = 0.0
    # Iterate over the training data
    for idx_sample, batch in enumerate(dataloader_train):
        if invert_targets:
            source = batch["B"].to(device)
            target = batch["A"].to(device)
        else:
            source = batch["A"].to(device)
            target = batch["B"].to(device)
        batch_size = target.size()[0]
        
        # Adversarial ground truths (you can do soft-label here....)
        # Remember that our discriminator outputs a grid of patches
        # So valid/fake will be a spatial tensor
        valid = torch.ones(batch_size, *patch).to(device)
        fake = torch.zeros(batch_size, *patch).to(device)
    
        # Train Generators
        optimizer_G.zero_grad()
        # Generate G(x) shape [batch, 3, 256, 256]
        G_x = G(source)
        pred_fake = D(G_x, source)
        
        loss_GAN = criterion_GAN(pred_fake, valid)
        
        # Pixel-wise loss
        loss_pixel = criterion_pixelwise(G_x, target)
        
        # Generator Loss
        loss_G = loss_GAN + lambda_pixel * loss_pixel
        loss_G.backward()
        optimizer_G.step()
        
        # Train Discriminator
        optimizer_D.zero_grad()
        # Real D(target | source) shape: [batch, 1, 16, 16]
        pred_real = D(target, source)
        loss_real = criterion_GAN(pred_real, valid)
        # Fake D(G(source) | source) shape: [batch, 1, 16, 16]
        pred_fake = D(G_x.detach(), source)
        loss_fake = criterion_GAN(pred_fake, fake)
        
        # Discriminator Loss
        loss_D = 0.5 * (loss_real + loss_fake)
        loss_D.backward()
        optimizer_D.step()
        
        # Update statistics
        running_loss_G += loss_G.item() * batch_size
        # Update statistics
        running_loss_D += loss_D.item() * batch_size
    
    # Epoch ends
    epoch_loss_generator = running_loss_G / len(dataloader_train.dataset)
    epoch_loss_discriminator = running_loss_D / len(dataloader_train.dataset)
    
    # Send results to tensorboard
    writer.add_scalar('train/loss_generator', epoch_loss_generator, epoch)
    writer.add_scalar('train/loss_discriminator', epoch_loss_discriminator, epoch)
    
    # Send images to tensorboard
    writer.add_images('train/source', source, epoch)
    writer.add_images('train/target', target, epoch)
    writer.add_images('train/G_x', G_x, epoch)
    writer.add_images('train/D_G_x', pred_fake, epoch)

 74%|███████▎  | 147/200 [1:10:16<25:20, 28.69s/it]